###Electric Vehicle Charging Forcasting 


ACN-Data was collected from two Adaptive Charging Networks
located in California. 

In [ ]:
#importing libraries
import numpy as np
import datetime
import pandas as pd
from matplotlib import pyplot

In [ ]:
df = pd.read_csv('data.csv',header=0, parse_dates=[0,1],usecols=(2,4,5))
df.columns = ['connectionTime','doneChargingTime','kWhDelivered']

NameError: ignored

In [ ]:
df.tail(5)

In [ ]:
# test = df.iloc[3082]
# a = test['connectionTime']
# b = test['doneChargingTime']
# print(a)
# print(b)
# print(b-a)
# a = pd.to_datetime(a.strftime('%m/%d/%Y'))
# print(a)
# print(type(a))

In [ ]:
# - pd.Timedelta(hours=1)
newData = []
for i in range(len(df)):
    # print('---------------------------------------------')
    data_to_insert = []
    item = df.iloc[i]
    start = item['connectionTime']
    end = item['doneChargingTime']
    kWhDelivered = item['kWhDelivered']

    # total difference between starting and ending time
    difference = end - start
    difference_seconds = difference.total_seconds()

    # if total difference in seconds is less than 0 , it means data is corrupt
    if (difference_seconds > 0 ):
        # print("difference: ",difference)
        # print("difference_seconds: ",difference_seconds)
        
        # absolute hour of starting and ending time and corresponding data 
        start_H = item['connectionTime'].replace(minute=0, second=0)
        end_H = item['doneChargingTime'].replace(minute=0, second=0)  

        ############# TRICK TO GET KWHDELIVERED PER HOUR ###############
        absolute_hours_bw = (end_H - start_H)  / np.timedelta64(1, 'h')  - 1     
        absolute_seconds_per_hour = 3600
        # print("absolute_hours_bw: ",absolute_hours_bw)

        # confirms that starting and ending time is not in same date and same hour
        if (absolute_hours_bw >=0):

            # first part corresponds to time in first hour of starting , ex. if start_time is ('10:10:00') then first part is 
            # between 10 and 11 which actually carries 50 minutes
            # same with second part as last hour , ex if end_time is ('10:10:00') then second part is between 10 and 11 
            # which carries 10 minutes
            # absolute part will be same as all hours carry 60 minutes, hence equal kwhdelivered for all
    
            seconds_in_first_part = 3600 - (start - start_H).total_seconds()
            seconds_in_second_part = (end - end_H).total_seconds()
            
            # kWhDelivered in three sections
            kWhDelivered_in_first_part = (seconds_in_first_part/difference_seconds) * kWhDelivered
            kWhDelivered_in_second_part = (seconds_in_second_part/difference_seconds) * kWhDelivered
            kWhDelivered_in_absolute_part = (absolute_seconds_per_hour/difference_seconds) * kWhDelivered

            # first part appended
            data_to_insert.append([start_H,kWhDelivered_in_first_part])

            # looping for absolute parts
            for j in range(int(absolute_hours_bw)):
                start_H = start_H + pd.Timedelta(hours=1)
                data_to_insert.append([start_H,kWhDelivered_in_absolute_part])

            # second part appended
            data_to_insert.append([end_H,kWhDelivered_in_second_part])
        # full kwhdelivered to same hour
        else:
            data_to_insert.append([start_H,kWhDelivered])
        newData += data_to_insert

In [ ]:
newdf = pd.DataFrame(newData)
newdf.columns = ['chargingHour','kWhDelivered']
newdf = newdf.set_index('chargingHour')
newdf
# for january to march
# newdf = newdf["2020-01-01 00:00:00+00:00":"2020-03-01 00:00:00+00:00"]


# for doing per day
df_day = newdf.resample('D').sum()

# for doing per hour
df_hour = newdf.groupby(['chargingHour']).sum()

df_week = newdf.resample('W').sum()


In [ ]:
df_day.tail(5)


In [ ]:
df_hour.tail(5)


In [ ]:
df_week.tail(5)

In [ ]:
# REMOVE OUTLIERS
def mod_outlier(df,col):
    df1 = df.copy()
    df = df._get_numeric_data()

    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    iqr = q3 - q1

    lower_bound = q1 -(1.5 * iqr) 
    upper_bound = q3 +(1.5 * iqr)
    print("lower_bound: ",lower_bound)
    print("upper_bound: ",upper_bound)
    
    for i in range(0,len(df[col])):
        if df[col][i] < lower_bound[col]:            
            df[col][i] = lower_bound[col]

        if df[col][i] > upper_bound[col]:            
            df[col][i] = upper_bound[col]  



In [ ]:
################### DAY WISE ########################
pyplot.figure(dpi=1200)
pyplot.plot(df_day.index, df_day['kWhDelivered'],label='original',linewidth=1)
pyplot.gcf().autofmt_xdate()
pyplot.title('Day Wise on Caltech Dataset')
pyplot.xlabel('Date')
pyplot.ylabel('Energy Delivered (in KWh)')
pyplot.savefig('Daywise.png', dpi=1200)

In [ ]:
################### HOUR WISE ########################
pyplot.figure(dpi=1200)
pyplot.plot(df_hour.index, df_hour['kWhDelivered'], label='original',linewidth=1)
pyplot.gcf().autofmt_xdate()
pyplot.title('Hour Wise on Caltech Dataset ')
pyplot.xlabel('Date')
pyplot.ylabel('Energy Delivered (in KWh)')
pyplot.savefig('Hourwise.png', dpi=1200)

In [ ]:
################### Week WISE ########################
pyplot.figure(dpi=1200)
pyplot.plot(df_week.index, df_week['kWhDelivered'], label='original',linewidth=1)
pyplot.gcf().autofmt_xdate()
pyplot.title('Week Wise on Caltech Dataset')
pyplot.xlabel('Date')
pyplot.ylabel('Energy Delivered')
pyplot.savefig('Weekwise.png', dpi=1200)

In [ ]:
################### CLEANING OUTLIERS ########################
mod_outlier(df_day,"kWhDelivered")
mod_outlier(df_hour,"kWhDelivered")
mod_outlier(df_week,"kWhDelivered")

In [ ]:
################### DAY WISE ########################
pyplot.figure(dpi=1200)
pyplot.plot(df_day.index, df_day['kWhDelivered'], label='original',linewidth=0.5)
pyplot.gcf().autofmt_xdate()
pyplot.title('Day Wise on Caltech Dataset')
pyplot.xlabel('Date')
pyplot.ylabel('Energy Delivered')
pyplot.savefig('DDaywise.png', dpi=1200)

In [ ]:
################### HOUR WISE ########################
pyplot.figure(dpi=1200)
pyplot.plot(df_hour.index, df_hour['kWhDelivered'], label='original',linewidth=0.5)
pyplot.gcf().autofmt_xdate()
pyplot.title('Hour Wise on Caltech Dataset')
pyplot.xlabel('Date')
pyplot.ylabel('Energy Delivered')
pyplot.savefig('DHourwise.png', dpi=1200)

In [ ]:
slide_hour = 24
slide_day = 7
slide_week = 6

In [ ]:
################### DAY WISE ########################

num_data = len(df_day)
num_train_day = int(0.80 * num_data)
num_test_day = num_data
train_data_day = df_day[0:num_train_day]
test_data_day = df_day[:]
test_day_index = df_day.index[slide_day:]


################### HOUR WISE ########################

num_data = len(df_hour)
num_train_hour = int(0.80 * num_data)
num_test_hour = num_data - num_train_hour
train_data_hour = df_hour[0:num_train_hour]
test_data_hour = df_hour[num_train_hour:]
test_hour_index = df_hour.index[num_train_hour+slide_hour:]


################### WEEK WISE ########################

num_data = len(df_week)
num_train_week = int(0.80 * num_data)
num_test_week = num_data
train_data_week = df_week[0:num_train_week]
test_data_week = df_week[:]
test_week_index = df_week.index[slide_week:]

Feature Scaling
We will normalising our data using MinMaxScaler 


In [ ]:
# Normalising the data using MinMaxScaler 
from sklearn.preprocessing import MinMaxScaler

################### DAY WISE ########################

sc_day = MinMaxScaler()
train_data_day = sc_day.fit_transform(train_data_day)
test_data_day = sc_day.transform(test_data_day)


################### HOUR WISE ########################

sc_hour = MinMaxScaler()
train_data_hour = sc_hour.fit_transform(train_data_hour)
test_data_hour = sc_hour.transform(test_data_hour)

################### WEEK WISE ########################

sc_week = MinMaxScaler()
train_data_week = sc_week.fit_transform(train_data_week)
test_data_week = sc_week.transform(test_data_week)



In [ ]:
################### DAY WISE ########################

# TRAINING DATA

X_train_day = []
y_train_day = []

for i in range (slide_day, num_train_day):
    X_train_day.append(train_data_day[i-slide_day:i, 0])
    y_train_day.append(train_data_day[i,0])
X_train_day, y_train_day = np.array(X_train_day), np.array(y_train_day)
# Reshaping
X_train_day_ann = X_train_day
X_train_day = np.reshape(X_train_day, (X_train_day.shape[0], X_train_day.shape[1], 1))



# TESTING DATA

X_test_day = []
y_test_day = []

for i in range (slide_day, num_test_day):
    X_test_day.append(test_data_day[i-slide_day:i, 0])
    y_test_day.append(test_data_day[i,0])
X_test_day, y_test_day = np.array(X_test_day), np.array(y_test_day)
# Reshaping
X_test_day_ann = X_test_day
X_test_day = np.reshape(X_test_day, (X_test_day.shape[0], X_test_day.shape[1], 1))
y_test_day = y_test_day.reshape(-1,1)


################### HOUR WISE ########################


# TRAINING DATA

X_train_hour = []
y_train_hour = []

for i in range (slide_hour, num_train_hour):
    X_train_hour.append(train_data_hour[i-slide_hour:i, 0])
    y_train_hour.append(train_data_hour[i,0])
X_train_hour, y_train_hour = np.array(X_train_hour), np.array(y_train_hour)
# Reshaping
X_train_hour_ann = X_train_hour
X_train_hour = np.reshape(X_train_hour, (X_train_hour.shape[0], X_train_hour.shape[1], 1))



# TESTING DATA

X_test_hour = []
y_test_hour = []

for i in range (slide_hour, num_test_hour):
    X_test_hour.append(test_data_hour[i-slide_hour:i, 0])
    y_test_hour.append(test_data_hour[i,0])
X_test_hour, y_test_hour = np.array(X_test_hour), np.array(y_test_hour)
# Reshaping
X_test_hour_ann = X_test_hour
X_test_hour = np.reshape(X_test_hour, (X_test_hour.shape[0], X_test_hour.shape[1], 1))
y_test_hour = y_test_hour.reshape(-1,1)



################### WEEK WISE ########################


# TRAINING DATA

X_train_week = []
y_train_week = []

for i in range (slide_week, num_train_week):
    X_train_week.append(train_data_week[i-slide_week:i, 0])
    y_train_week.append(train_data_week[i,0])
X_train_week, y_train_week = np.array(X_train_week), np.array(y_train_week)
# Reshaping
X_train_week_ann = X_train_week
X_train_week = np.reshape(X_train_week, (X_train_week.shape[0], X_train_week.shape[1], 1))



# TESTING DATA

X_test_week = []
y_test_week = []

for i in range (slide_week, num_test_week):
    X_test_week.append(test_data_week[i-slide_week:i, 0])
    y_test_week.append(test_data_week[i,0])
X_test_week, y_test_week = np.array(X_test_week), np.array(y_test_week)
# Reshaping
X_test_week_ann = X_test_week
X_test_week = np.reshape(X_test_week, (X_test_week.shape[0], X_test_week.shape[1], 1))
y_test_week = y_test_week.reshape(-1,1)

In [ ]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional

Model that we will be using is LSTM Recurrent Neural Networks for Time Series Prediction. We will be using 4 layers of LSTM and one Dense for output. Optimizer as adam and loss as mean square Error.


In [ ]:
# ######################## DAY WISE using RNN #################################
# Adding the first LSTM layer and some Dropout regularisation
model_day = Sequential()

# # Initialising the RNN
 
# Adding the first LSTM layer and some Dropout regularisation
model_day.add(Bidirectional(LSTM(units = 16, return_sequences = True, input_shape = (X_train_day.shape[1], 1))))
model_day.add(Dropout(0.20))
 
model_day.add(LSTM(units = 32))
model_day.add(Dropout(0.20))
# Adding the output layer
model_day.add(Dense(units = 1))
 
# Compiling the RNN
model_day.compile(optimizer = 'adam', loss = 'mean_squared_error')
 
# Fitting the RNN to the Training set
model_day.fit(X_train_day, y_train_day, epochs = 200, batch_size = 30, validation_data=(X_test_day,y_test_day))
model_day.summary()

In [ ]:
######################## DAY WISE using ANN #################################

# Initialising the ANN
model_day_ann = Sequential()
 
model_day_ann.add(Dense(units = 6, activation = 'relu',input_shape= X_train_day_ann.shape))

#adding the second hidden layer
model_day_ann.add(Dense(units = 6,activation = 'relu'))
 
#adding the output layer
model_day_ann.add(Dense(units = 1,activation = 'sigmoid'))

model_day_ann.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the ANN to the Training set
model_day_ann.fit(X_train_day_ann, y_train_day, epochs = 200, batch_size = 30, validation_data=(X_test_day_ann,y_test_day))
model_day_ann.summary()

In [ ]:
predicted_energy_day = model_day.predict(X_test_day)
predicted_energy_day = sc_day.inverse_transform(predicted_energy_day)
test_energy_day = sc_day.inverse_transform(y_test_day)

In [ ]:
predicted_energy_day_ann = model_day_ann.predict(X_test_day_ann)
predicted_energy_day_ann = sc_day.inverse_transform(predicted_energy_day_ann)

In [ ]:
pyplot.figure(dpi=1000)
pyplot.plot(test_day_index,test_energy_day, color = 'red', label = 'Actual Energy Required',linewidth=0.7)
pyplot.plot(test_day_index,predicted_energy_day, color = 'orange', label = 'Predicted Energy Required RNN',linewidth=0.7)
pyplot.plot(test_day_index,predicted_energy_day_ann, color = 'blue', label = 'Predicted Energy Required using ANN',linewidth=0.7)
pyplot.gcf().autofmt_xdate()
pyplot.title('EV charging Forcasting comparision')
pyplot.xlabel('Time')
pyplot.ylabel('Energy Required (in KWh)')
pyplot.legend()
pyplot.show()
pyplot.savefig('XDaywise.png', dpi=1200)

In [ ]:
from sklearn.metrics import r2_score
r2_score(test_energy_day, predicted_energy_day)

In [ ]:
r2_score(test_energy_day, predicted_energy_day_ann)

Not Much difference in Accuracy of ANN and RNN this case.

                                                    HOUR WISE
                                                    

In [ ]:
######################## HOUR WISE using RNN #################################
# Initialising the RNN
model_hour = Sequential()
 
# Adding the first LSTM layer and some Dropout regularisation
model_hour.add(Bidirectional(LSTM(units = 16, return_sequences = True, input_shape = (X_train_hour.shape[1], 1))))
model_hour.add(Dropout(0.2))
 

# Adding a third LSTM layer and some Dropout regularisation
model_hour.add(LSTM(units = 32))
model_hour.add(Dropout(0.2))
 
# Adding the output layer
model_hour.add(Dense(units = 1))
 
# Compiling the RNN
model_hour.compile(optimizer = 'adam', loss = 'mean_squared_error')
 
# Fitting the RNN to the Training set
model_hour.fit(X_train_hour, y_train_hour, epochs = 250, batch_size = 128,validation_data=(X_test_hour,y_test_hour))
model_hour.summary()

In [ ]:
######################## HOUR WISE using ANN #################################

# Initialising the ANN
model_hour_ann = Sequential()
 
model_hour_ann.add(Dense(units =  6, activation = 'relu',input_shape= X_train_hour_ann.shape))

#adding the second hidden layer
model_hour_ann.add(Dense(units = 6,activation = 'relu'))
 
#adding the output layer
model_hour_ann.add(Dense(units = 1,activation = 'sigmoid'))

model_hour_ann.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the ANN to the Training set
model_hour_ann.fit(X_train_hour_ann, y_train_hour, epochs = 200, batch_size = 30, validation_data=(X_test_hour_ann,y_test_hour))
model_hour_ann.summary()

In [ ]:
predicted_energy_hour = model_hour.predict(X_test_hour)
predicted_energy_hour = sc_hour.inverse_transform(predicted_energy_hour)
test_energy_hour = sc_hour.inverse_transform(y_test_hour)

In [ ]:
predicted_energy_hour_ann = model_hour_ann.predict(X_test_hour_ann)
predicted_energy_hour_ann = sc_hour.inverse_transform(predicted_energy_hour_ann)

In [ ]:
pyplot.figure(dpi=1000)
pyplot.plot(test_hour_index,predicted_energy_hour, color = 'blue', label = 'Predicted Energy Required with LSTM' ,linewidth=0.6)
pyplot.plot(test_hour_index,predicted_energy_hour_ann, color = 'orange', label = 'Predicted Energy Required with ANN',linewidth=0.6)
pyplot.plot(test_hour_index,test_energy_hour, color = 'red', label = 'Actual Energy Required',linewidth=0.6)
pyplot.gcf().autofmt_xdate()
pyplot.title('EV charging Forcasting comparision')
pyplot.xlabel('Time')
pyplot.ylabel('Energy Required (in KWh)')
pyplot.legend()
pyplot.show()
pyplot.savefig('XHourwise.png', dpi=1200)

In [ ]:
r2_score(test_energy_hour, predicted_energy_hour)

In [ ]:
r2_score(test_energy_hour, predicted_energy_hour_ann)

RNN seems to be more effective in hourly prediction.

In [ ]:
######################## Week WISE using RNN #################################
# Initialising the RNN
model_week = Sequential()
 
# Adding the first LSTM layer and some Dropout regularisation
model_week.add(Bidirectional(LSTM(units = 16, return_sequences = True, input_shape = (X_train_week.shape[1], 1))))
model_week.add(Dropout(0.2))
 

# Adding a third LSTM layer and some Dropout regularisation
model_week.add(LSTM(units = 32))
model_week.add(Dropout(0.2))
 
# Adding the output layer
model_week.add(Dense(units = 1))
 
# Compiling the RNN
model_week.compile(optimizer = 'adam', loss = 'mean_squared_error')
 
# Fitting the RNN to the Training set
model_week.fit(X_train_week, y_train_week, epochs = 250, batch_size = 128,validation_data=(X_test_week,y_test_week))
model_week.summary()

In [ ]:
######################## Week WISE using ANN #################################

# Initialising the ANN
model_week_ann = Sequential()
 
model_week_ann.add(Dense(units =  6, activation = 'relu',input_shape= X_train_week_ann.shape))

#adding the second hidden layer
model_week_ann.add(Dense(units = 6,activation = 'relu'))
 
#adding the output layer
model_week_ann.add(Dense(units = 1,activation = 'sigmoid'))

model_week_ann.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the ANN to the Training set
model_week_ann.fit(X_train_week_ann, y_train_week, epochs = 200, batch_size = 30, validation_data=(X_test_week_ann,y_test_week))
model_week_ann.summary()

In [ ]:
predicted_energy_week = model_week.predict(X_test_week)
predicted_energy_week = sc_week.inverse_transform(predicted_energy_week)
test_energy_week = sc_week.inverse_transform(y_test_week)

In [ ]:
predicted_energy_week_ann = model_week_ann.predict(X_test_week_ann)
predicted_energy_week_ann = sc_week.inverse_transform(predicted_energy_week_ann)

In [ ]:
pyplot.figure(dpi=720)
pyplot.plot(test_week_index,predicted_energy_week, color = 'blue', label = 'Predicted Energy Required with LSTM',linewidth=0.7)
pyplot.plot(test_week_index,predicted_energy_week_ann, color = 'orange', label = 'Predicted Energy Required with ANN',linewidth=0.7)
pyplot.plot(test_week_index,test_energy_week, color = 'red', label = 'Actual Energy Required',linewidth=0.7)
pyplot.gcf().autofmt_xdate()
pyplot.title('EV charging Forcasting comparision')
pyplot.xlabel('Time')
pyplot.ylabel('Energy Required (in KWh)')
pyplot.legend()
pyplot.show()
pyplot.savefig('XWeekwise.png', dpi=1200)

In [ ]:
r2_score(test_energy_week, predicted_energy_week)

In [ ]:
r2_score(test_energy_week, predicted_energy_week_ann)

Not a good prediction due to low range of dataset for weekly prediction.